# Build a Neural Network #

Neural networks comprise of layers/modules that perform operations on data. The torch.nn namespace provides all the building blocks you need to build your own neural network. Every module in PyTorch subclasses the nn.Module. A neural network is a module itself that consists of other modules (layers). This nested structure allows for building and managing complex architectures easily.

In [1]:
import os
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms

In [2]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print('Using {} device'.format(device))

Using cuda device


In [3]:
# We define our neural network by subclassing nn.Module, and initialize the neural network layers in __init__. 
# Every nn.Module subclass implements the operations on input data in the forward method.

class NeuralNetwork(nn.Module):
    def __init__(self, input_width, input_height, label_dim):
        super(NeuralNetwork, self).__init__()
        self.flatten = nn.Flatten() # Used to convert the 2D input into 1D (28x28 to 784)
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(input_width*input_height, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, label_dim),
            nn.ReLU()
        )
        self.softmax = nn.Softmax(dim=1)
        self.softmax_result = 0

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        self.softmax_result = self.softmax(logits)
        return logits

### One by one the modules ###


In [4]:
input_image = torch.rand(3,28,28) # 3 is used to exemplify a mini-batch
print(input_image.size())

torch.Size([3, 28, 28])


In [5]:
# We initialize the nn.Flatten layer to convert each 2D 28x28 
# image into a contiguous array of 784 pixel values ( the minibatch dimension (at dim=0) is maintained).

flatten = nn.Flatten()
flat_image = flatten(input_image)
print(flat_image.size())

torch.Size([3, 784])


In [6]:
layer1 = nn.Linear(in_features=28*28, out_features=20) # Linear transformation
hidden1 = layer1(flat_image)
print(hidden1.size())

torch.Size([3, 20])


Non-linear activations are what create the complex mappings between the model’s inputs and outputs. They are applied after linear transformations to introduce nonlinearity, helping neural networks learn a wide variety of phenomena.

In this model, we use nn.ReLU between our linear layers, but there’s other activations to introduce non-linearity in your model such as LeakyReLU, Sigmoid, TANH, etc.

In [7]:
print(f"Before ReLU: {hidden1}\n\n")
hidden1 = nn.ReLU()(hidden1)
print(f"After ReLU: {hidden1}")

Before ReLU: tensor([[ 1.7573e-02,  4.1694e-01, -6.7020e-02, -9.7063e-03, -6.4914e-02,
          3.9334e-01,  4.5783e-02,  2.9296e-01,  5.6483e-01, -2.8356e-01,
          8.3396e-02,  5.7822e-01,  2.4769e-01, -6.5515e-01, -1.9853e-01,
          1.0206e+00,  6.0045e-02, -7.1862e-04, -2.0854e-01, -9.5183e-02],
        [ 3.0395e-01,  5.8897e-01, -8.0327e-02, -7.1482e-03, -3.9476e-01,
          2.1905e-01, -9.7230e-02,  1.9060e-01,  9.7130e-02, -2.5420e-02,
         -8.3685e-02,  8.0775e-01,  4.0999e-01, -8.6468e-01, -2.2341e-01,
          6.3974e-01,  3.4347e-01,  3.7644e-01,  1.3201e-01, -1.3214e-01],
        [ 9.7511e-02,  5.5372e-01,  2.7525e-01,  3.4343e-02,  5.8041e-02,
          1.5501e-01, -3.9706e-01, -1.5643e-01,  2.0734e-01, -2.7891e-01,
         -6.4181e-02,  6.8991e-01,  3.9327e-01, -5.5614e-01,  2.0688e-01,
          2.7230e-01, -1.3768e-01,  4.9278e-02, -4.7706e-02,  9.8192e-02]],
       grad_fn=<AddmmBackward>)


After ReLU: tensor([[0.0176, 0.4169, 0.0000, 0.0000, 0.0000, 

***
nn.Sequential is an ordered container of modules. The data is passed through all the modules in the same order as defined. You can use sequential containers to put together a quick network like seq_modules.

In [8]:
seq_modules = nn.Sequential(
    flatten,
    layer1,
    nn.ReLU(),
    nn.Linear(20, 10)
)
input_image = torch.rand(3,28,28)
logits = seq_modules(input_image)
print(logits)

tensor([[ 0.1572,  0.3569,  0.1734, -0.2430, -0.1336,  0.0281, -0.0052,  0.3756,
          0.3543, -0.1139],
        [ 0.0756,  0.2918,  0.0968, -0.2669, -0.1232, -0.0526, -0.0187,  0.4116,
          0.3053, -0.0704],
        [ 0.1789,  0.1703,  0.1015, -0.2708, -0.2091, -0.0394, -0.1522,  0.3546,
          0.4009, -0.0705]], grad_fn=<AddmmBackward>)


***
Finally, we use the nn.softmax, which scale the logits values to [0, 1] from [-infty. infty], representing the models predicted probabilties.

`dim` indicates which dimension should softmax be applied (dim=1 so we don't mess with the minibatch)

In [9]:
softmax = nn.Softmax(dim=1)
pred_probab = softmax(logits)
print(pred_probab)

tensor([[0.1041, 0.1271, 0.1058, 0.0697, 0.0778, 0.0915, 0.0885, 0.1295, 0.1267,
         0.0794],
        [0.0990, 0.1229, 0.1011, 0.0703, 0.0811, 0.0871, 0.0901, 0.1385, 0.1245,
         0.0855],
        [0.1114, 0.1105, 0.1031, 0.0711, 0.0756, 0.0896, 0.0800, 0.1328, 0.1391,
         0.0868]], grad_fn=<SoftmaxBackward>)


In [11]:
model = NeuralNetwork(28, 28, 10).to(device)
print(model) # You can see the networks structure

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
  (softmax): Softmax(dim=1)
)


In [12]:
print("Model structure: ", model, "\n\n")

for name, param in model.named_parameters():
    print(f"Layer: {name} | Size: {param.size()} | Values : {param[:2]} \n")

Model structure:  NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
  )
  (softmax): Softmax(dim=1)
) 


Layer: linear_relu_stack.0.weight | Size: torch.Size([512, 784]) | Values : tensor([[ 0.0010,  0.0033,  0.0177,  ...,  0.0127,  0.0292, -0.0290],
        [-0.0174,  0.0110, -0.0302,  ..., -0.0294,  0.0299,  0.0127]],
       device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.0.bias | Size: torch.Size([512]) | Values : tensor([0.0060, 0.0191], device='cuda:0', grad_fn=<SliceBackward>) 

Layer: linear_relu_stack.2.weight | Size: torch.Size([512, 512]) | Values : tensor([[-0.0298,  0.0357,  0.0310,  ...,  0.0256, -0.0139, -0.0340],
        [ 0.0189,  0.0280, -0.0174,  ..., -0.0019, -0.0233,  0.0

### Use the model ###

In [23]:
X = torch.rand(1, 28, 28, device=device)
logits = model(X)
pred_probab = nn.Softmax(dim=1)(logits)
print(pred_probab)
y_pred = pred_probab.argmax(1)
print(f"Predicted class: {y_pred}")

print(model.softmax_result)
pred_probab = model.softmax_result.argmax(1)  # Done directly in the network?
print(f"Predicted class: {y_pred}")


tensor([[0.0984, 0.1063, 0.0955, 0.0998, 0.1063, 0.0993, 0.0955, 0.0970, 0.0955,
         0.1065]], device='cuda:0', grad_fn=<SoftmaxBackward>)
Predicted class: tensor([9], device='cuda:0')
tensor([[0.0984, 0.1063, 0.0955, 0.0998, 0.1063, 0.0993, 0.0955, 0.0970, 0.0955,
         0.1065]], device='cuda:0', grad_fn=<SoftmaxBackward>)
Predicted class: tensor([9], device='cuda:0')
